In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
num_breeds = 10
# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
train_generator = train_datagen.flow_from_directory('path/to/train_data', target_size=(224, 224), batch_size=32, class_mode='categorical')

# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1000, activation='relu')(x)  # Adjust the number of units based on the number of breeds
predictions = Dense(num_breeds, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Make predictions
new_image = load_image('path/to/new_image.jpg')
predictions = model.predict(new_image)
predicted_breed = np.argmax(predictions)
print("Predicted breed:", breed_labels[predicted_breed])

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/train_data'

In [ ]:
pip install tensorflow keras kaggle opencv-python


In [ ]:
kaggle datasets download -d jessicali9530/stanford-dogs-dataset


SyntaxError: invalid syntax (<ipython-input-2-132b9b85a66a>, line 1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define directories
base_dir = 'path_to_stanford_dogs_dataset'

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'valid')

# Image Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential()

# First Convolutional Layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Second Convolutional Layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Third Convolutional Layer
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Fully Connected Layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(120, activation='softmax'))  # 120 dog breeds

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load an image from the test set
img_path = 'path_to_image.jpg'
img = image.load_img(img_path, target_size=(150, 150))

# Preprocess the image
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

# Make a prediction
predictions = model.predict(img_tensor)
predicted_class = np.argmax(predictions, axis=1)

# Get the dog breed label
labels = train_generator.class_indices
breed = list(labels.keys())[list(labels.values()).index(predicted_class[0])]

print(f"Predicted dog breed: {breed}")


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models

# Load the Stanford Dogs dataset
dataset_name = 'stanford_dogs'
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name,
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Preprocess function: resize and normalize
IMG_SIZE = 224
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0  # Normalize the pixel values
    return image, label

ds_train = ds_train.map(preprocess).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(ds_info.features['label'].num_classes, activation='softmax')  # 120 breeds
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(ds_train, epochs=10, validation_data=ds_test)

# Evaluate the model
test_loss, test_acc = model.evaluate(ds_test)
print(f"Test accuracy: {test_acc}")

# Perform inference on an image
def predict_breed(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    predictions = model.predict(img_array)
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]

    breed_names = ds_info.features['label'].int2str(predicted_class)
    return breed_names

# Example: Perform inference on a sample image
image_path = '/content/download.jpeg'
breed = predict_breed(image_path)
print(f"The predicted breed is: {breed}")
